In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
import psycopg2
from psycopg2 import sql

SyntaxError: invalid syntax (3085632279.py, line 8)

In [2]:
logging.basicConfig(filename='etl_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def get_data(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36"
    }

    try:
        r = requests.get(url=url, headers=headers)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error in making the request: {e}")
        return None

    soup = BeautifulSoup(r.text, "lxml")

    table_all = soup.find_all("table", class_="wikitable sortable mw-collapsible")

    if len(table_all) < 2:
        print("Error: Table not found.")
        return None

    value_tr = table_all[1].find("tbody")
    rows = value_tr.find_all('td')

    def clean_text(text):
        if "," in text:
            return text.replace("\n", "").replace(".","").replace(",", ".")
        else:
            return text.replace("\n", "").replace("\xa0 ", "").replace(",", ".")

    data = [(clean_text(rows[i].text), clean_text(rows[i + 1].text), round(float(clean_text(rows[i + 2].text)),2))
            for i in range(0, len(rows), 3)]

    final_df = pd.DataFrame(data, columns=['Rank', 'Bank_Name', 'US']).astype({'Rank': int, 'Bank_Name': str,'US': float})
    return final_df

In [4]:
def load_money():
    
    base_currency = "USD"

    url = f"https://open.er-api.com/v6/latest/{base_currency}"

    params = {"base": base_currency, "symbols": "EUR,GBP,INR"}

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        final_data = {"EUR" : data['rates']["EUR"], "GBP" : data['rates']["GBP"], "INR" : data['rates']["INR"]}
        return final_data

    else:
        print(f"Error: {response.status_code}")
        return None

In [5]:
def convert(money, value_money, types = None):
    try:
        return round(float(money * value_money[types]),2)
    
    except requests.exceptions.RequestException as e:
        print(f"Error in making the request: {e}")
        return None

In [6]:
def transform(prep_data):
    conversion_rates = load_money()
    for currency_type in ["EUR", "GBP", "INR"]:
        prep_data[currency_type] = prep_data['US'].apply(lambda x: convert(x, conversion_rates, types = currency_type))
    return prep_data

In [7]:
db_params = {
    'host': '127.0.0.1',
    'database': 'microecon',
    'user': 'postgres',
    'password': '1111',
}

db_name = "bank"

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {db_name} (
        rank INTEGER PRIMARY KEY,
        Bank_Name CHARACTER VARYING(50),
        US  DOUBLE PRECISION,
        EUR  DOUBLE PRECISION,
        GBP  DOUBLE PRECISION,
        INR  DOUBLE PRECISION
    );
"""

In [8]:
def load_to_db(tr_data):
    data_tuples = [tuple(row) for row in tr_data.values]
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    cursor.execute(create_table_query)
    connection.commit()
    insert_query = sql.SQL(f"INSERT INTO {db_name} ({', '.join(tr_data.columns)}) VALUES ({', '.join(['%s' for _ in tr_data.columns])})")
    cursor.executemany(insert_query, data_tuples)
    connection.commit()
    cursor.close()
    connection.close()

In [9]:
def test_db(query):
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()

    return result

In [10]:
bank_data = get_data("https://en.wikipedia.org/wiki/List_of_largest_banks")
logging.info('Data load.')
trans_data = transform(bank_data)
logging.info('Data transform.')
load_to_db(trans_data)
logging.info('Load in bd.')
bank_data.to_csv('bank_data', index = False)
logging.info('Save in csv.')
print(test_db("SELECT * FROM bank"))
logging.info('All value in bd.')

UniqueViolation: ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "bank_pkey"
DETAIL:  Ключ "(rank)=(1)" уже существует.
